In [1]:
import numpy as np
from numpy import pi
from numpy.linalg import norm
from matplotlib import pyplot as plt
from scipy import signal
from scipy.ndimage import gaussian_filter

import simulation
from algorithm import time_delay_estimation, far_locate

## 仿真实验


### 声场仿真

设置参数


In [2]:
f = 37500  # 声源频率
T = 1  # 采样时长
r = 300  # 距离
theta = np.deg2rad(60)  # 角度

d = 0.5
K = 0.6 / d
c = 1500  # 声速
fs = 500e3  # 采样频率500k

#### 几何模型

In [3]:
d1, d2, d3 = (K + 1) * d / 2, (K - 1) * d / 2, (K + 1) * d / 2
S = np.array([r * np.cos(theta), r * np.sin(theta)])  # FIXME: 这x该是-, 方位近似解需要重新推导一下

r1 = float(norm(S - [-d1, 0]))
r2 = float(norm(S - [-d2, 0]))
r3 = float(norm(S - [d3, 0]))

# 真实时延
t12 = (r1 - r2) / c
t23 = (r2 - r3) / c

#### 生成各阵元电信号AD采样后信号

In [4]:
# 三阵元处采样信号序列 # TODO: 改成用同一个函数生成, 包含阵元的坐标, 生成信号矩阵
weights = [2, 2]
x1 = simulation.sig_gen(c, f, r1, fs, T, weights, 0)
x2 = simulation.sig_gen(c, f, r2, fs, T, weights, 1000)
x3 = simulation.sig_gen(c, f, r3, fs, T, weights, 1000000)

##### 信号图像

In [5]:
# plt.plot(x1)
# plt.plot(x2)
# plt.plot(x3)
# plt.show()

### 定位

#### 时延估计

In [6]:
period, T_on = 1, 10e-3
t = np.arange(len(x1)) / fs
s_ref = np.cos(2 * pi * f * t) * (t % period < T_on)

tau1_frac = time_delay_estimation(x1, s_ref, f, fs)
tau2_frac = time_delay_estimation(x2, s_ref, f, fs)
tau3_frac = time_delay_estimation(x3, s_ref, f, fs)

tau12_hat = tau1_frac - tau2_frac
tau23_hat = tau2_frac - tau3_frac

In [7]:
phi12_hat, phi23_hat = tau12_hat * f, tau23_hat * f

In [8]:
print(f'真实相位差phi12: {(phi12:=t12*f)}\n相关相位差phi12\': {phi12_hat}\n差异: {np.abs(phi12 - phi12_hat) / phi12 * 100:.2f}%')
print(f'真实相位差phi23: {(phi23:=t23*f)}\n相关相位差phi23\': {phi23_hat}\n差异: {np.abs(phi23 - phi23_hat) / phi23 * 100:.2f}%')

真实相位差phi12: 6.259366343141437
相关相位差phi12': 6.300000000000056
差异: 0.65%
真实相位差phi23: 7.4906163261488246
相关相位差phi23': 8.475000000000149
差异: 13.14%


#### 解算方位

In [9]:
r_e, theta_e = far_locate(phi12_hat / f, phi23_hat / f, c, K, d)

# TODO: 解决距离估计因为精度趋近0或inf问题
print(f'真实方位角: \t{theta/pi*180}度\n估计方位角: \t{theta_e/pi*180}度\n绝对误差: \t{theta/pi*180 - theta_e/pi*180}度\n相对误差: \t{(theta - theta_e)/theta * 100}%')
# print(algorithm.far_locate(t12, t23, c, K, d))

真实方位角: 	60.0度
估计方位角: 	57.50182614664359度
绝对误差: 	2.4981738533564126度
相对误差: 	4.163623088927347%
